In [2]:
# define imports
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
#import pandas as pd
#import PIL
import os, random

from glob import glob
from mpl_toolkits.axes_grid1 import ImageGrid
from IPython.display import display, Image
from IPython.core.display import HTML



import copy
#import cv2                

from io import open
import json
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os, os.path, random
from PIL import Image
import requests
import shutil
import time

import torch
from torch.autograd import Variable
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms


%matplotlib inline


# my imports

from os import listdir
from os.path import isfile, join, abspath
from style_transfer.stylize import transfer


In [3]:
# All images are resized to 224x224 and normalized
# Only training images receive further augmentation

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),    
}


In [4]:
data_dir = 'dogImages_style_transfer'
print(os.path.join(data_dir, 'valid'))


# we create some dictionaries
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'valid', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=5,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'valid', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid', 'test']}
print(dataset_sizes)
class_names = image_datasets['train'].classes
n_classes = len(class_names)

dogImages_style_transfer/valid
{'train': 59800, 'valid': 835, 'test': 836}


In [5]:
#import pandas as pd
import boto3
import sagemaker

In [6]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

In [6]:
import os

# this is the S3 data_dir name
data_dir = 'dogImages'
train_dir = 'train'

# set prefix, a descriptive name for a directory  
prefix = 'capstone-project'

# upload data for the first time; otherwise use the second expression (because the S3 upload takes some time)
#train_location = sagemaker_session.upload_data(os.path.join(data_dir), key_prefix=prefix)
train_location = 's3://{}/{}/'.format(bucket, prefix) 

In [9]:
# your import and estimator code, here
from sagemaker.pytorch import PyTorch

output_path = 's3://{}/{}'.format(bucket, prefix)

# note: this PyTorch estimator is specified with code that only
# uses the style transferred pictures, but no imgaug preprocessing!
source_dir = 'source_pytorch_style_transfer'

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir=source_dir,
                    role=role,
                    framework_version='1.1.0',
                    train_instance_count=1,
                    #train_instance_type='ml.c4.xlarge',
                    train_instance_type='ml.p2.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'epochs': 10
                    })


In [10]:
%%time

# Train your estimator on S3 training data
estimator.fit({'train': train_location})

2019-08-05 11:30:00 Starting - Starting the training job...
2019-08-05 11:30:02 Starting - Launching requested ML instances...
2019-08-05 11:30:40 Starting - Insufficient capacity error from EC2 while launching instances, retrying!......
2019-08-05 11:31:39 Starting - Preparing the instances for training.........
2019-08-05 11:33:11 Downloading - Downloading input data.............................................
2019-08-05 11:41:02 Training - Training image download completed. Training in progress..
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-08-05 11:41:03,687 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-08-05 11:41:03,714 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-08-05 11:41:06,732 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-08-05 11:41:07,061 sagemaker-containers INFO     Modul

Epoch: 1 #011Training Loss: 3.636029 #011Validation Loss: 1.318098
Epoch: 2 #011Training Loss: 2.686004 #011Validation Loss: 0.844991
Epoch: 3 #011Training Loss: 2.387923 #011Validation Loss: 0.685585
Epoch: 4 #011Training Loss: 2.232464 #011Validation Loss: 0.606484
Epoch: 5 #011Training Loss: 2.129745 #011Validation Loss: 0.561718
Epoch: 6 #011Training Loss: 2.060371 #011Validation Loss: 0.529383
Epoch: 7 #011Training Loss: 2.005694 #011Validation Loss: 0.505579
Epoch: 8 #011Training Loss: 1.954784 #011Validation Loss: 0.488833
Epoch: 9 #011Training Loss: 1.920151 #011Validation Loss: 0.476198
Epoch: 10 #011Training Loss: 1.887669 #011Validation Loss: 0.465194
2019-08-05 13:30:44,285 sagemaker-containers INFO     Reporting training SUCCESS

2019-08-05 13:30:45 Uploading - Uploading generated training model
2019-08-05 13:32:00 Completed - Training job completed
Billable seconds: 7130
CPU times: user 12.9 s, sys: 551 ms, total: 13.5 s
Wall time: 2h 2min 25s


In [11]:
%%time

# importing PyTorchModel
from sagemaker.pytorch import PyTorchModel

print(estimator.model_data)

# Create a model from the trained estimator data
# And point to the prediction script
model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='1.1.0',
                     entry_point='predict.py',
                     source_dir=source_dir)


s3://sagemaker-eu-central-1-505649883860/capstone-project/sagemaker-pytorch-2019-08-05-11-30-00-513/output/model.tar.gz
CPU times: user 18.8 ms, sys: 100 µs, total: 18.9 ms
Wall time: 94.9 ms


In [12]:
# ATTENTION: Please use the ml.t2.large instance here. Otherwise the deployment will fail because of resource constraints
# predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.large')



---------------------------------------------------------------------------------------------------------------!

In [13]:
def test_acc(test_set, predictor):
    ''' 
    This method calculates our accuracy metric. For this, it will send 
    batches of data to the endpoint and receive the predictions
    '''
    test_acc = 0.0
    running_corrects = 0

    for inputs, labels in test_set:
        print(labels)
        test_y_preds = predictor.predict(inputs)

        _, preds_tensor = torch.max(torch.from_numpy(test_y_preds), 1)
        preds = np.squeeze(preds_tensor.numpy()) if not use_cuda else np.squeeze(preds_tensor.cpu().numpy())

        print(f"preds is {preds}")
    
        # compare the batch of predictions with the ground truth of labels
        running_corrects += torch.sum(torch.from_numpy(preds) == labels)

    test_acc = running_corrects.double() / len(test_set.dataset)

    print('Acc: {:.2f}%'.format(test_acc*100))

    return test_acc

use_cuda = torch.cuda.is_available()
testacc=test_acc(iter(dataloaders['test']), predictor)
print(testacc)

tensor([ 5, 10, 33, 67, 94])
preds is [122  10  33  67  94]
tensor([ 17,  72, 112,  64, 113])
preds is [ 17  62 112  64 113]
tensor([70, 42, 44, 60, 64])
preds is [ 70  66 117  60  64]
tensor([126,  80,   6,  71, 119])
preds is [126  80   6  71 119]
tensor([ 62, 123,  39,  70,   4])
preds is [ 62 123  39  70   4]
tensor([37, 14, 73,  2, 81])
preds is [ 37  14  25   2 124]
tensor([45, 43, 31, 35, 93])
preds is [45 43 31 35 93]
tensor([ 94,  43,   4,  85, 111])
preds is [ 78  43   4  85 111]
tensor([106,  74,  32,  33,  53])
preds is [106  74  32  33  53]
tensor([ 12,  87,  33,   7, 105])
preds is [ 12  87  33   7 105]
tensor([ 68, 124,  10,  91,  56])
preds is [ 68 124  42  91  56]
tensor([61, 78, 70, 81, 18])
preds is [61 78 70 81 18]
tensor([  3,  48,   5,  61, 130])
preds is [  3  48   5  61 130]
tensor([  2,   8,  46, 119,  88])
preds is [  2  34  46 119  88]
tensor([122,  16, 114,  44,  77])
preds is [122  16 114  44 121]
tensor([ 38, 110,  41,  42,  37])
preds is [ 38 110  41  42 

preds is [ 57  19  14  98 120]
tensor([102,  63,  38,  31,  75])
preds is [40 63 38 31 75]
tensor([29, 48, 28, 50, 22])
preds is [29 48 28 50 22]
tensor([ 71,  91,  91,   8, 116])
preds is [121  91  91  87 116]
tensor([67, 86, 37, 46, 16])
preds is [105  86  37  46  16]
tensor([ 20,  69, 120,  76,  88])
preds is [ 20  69 120  76  16]
tensor([128,   7,  35,  43,  40])
preds is [75  7 35 43 40]
tensor([ 22,  64,  45, 132,  23])
preds is [ 22  79  45 132  23]
tensor([ 77,  68,  96,  40, 127])
preds is [ 77  68   2  40 127]
tensor([86, 29, 20, 47, 75])
preds is [86 29 20 47 75]
tensor([129,  82,  55,  52,  69])
preds is [129  82  55  63  69]
tensor([31, 44, 92, 72, 62])
preds is [31 44 92 36 67]
tensor([ 52,   1,   3,  11, 130])
preds is [52  1  3 11 72]
tensor([ 23,  95, 111,  90,   2])
preds is [ 23  95 111  90   2]
tensor([53, 72, 95, 18, 17])
preds is [ 53 130  95  18  17]
tensor([  9,  84, 102,  66,  75])
preds is [  9  84 102  83  75]
tensor([62,  7, 28, 89, 10])
preds is [ 62   7  2

Finally we want to see how our baseline model keeps up. For this, we first need to upload the original dogImages (no style transferred images includes)

In [7]:
import os

# this is the S3 data_dir name
data_dir = 'dogImages'

# set prefix, a descriptive name for a directory  
prefix = 'capstone-project-orig-data'

# upload data for the first time; otherwise use the second expression (because the S3 upload takes some time)
#train_location_orig_data = sagemaker_session.upload_data(os.path.join(data_dir), key_prefix=prefix)
train_location_orig_data = 's3://{}/{}/'.format(bucket, prefix) 

The estimator itself and the deployment is exactly the same. What a great benefit of our structured SageMaker deployment!

In [8]:
# your import and estimator code, here
from sagemaker.pytorch import PyTorch

output_path = 's3://{}/{}'.format(bucket, prefix)

# note: this PyTorch estimator is specified with code that only
# uses the style transferred pictures, but no imgaug preprocessing!
source_dir = 'source_pytorch_style_transfer'

# instantiate a pytorch estimator
estimator = PyTorch(entry_point='train.py',
                    source_dir=source_dir,
                    role=role,
                    framework_version='1.1.0',
                    train_instance_count=1,
                    #train_instance_type='ml.c4.xlarge',
                    train_instance_type='ml.p2.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'epochs': 10
                    })



In [9]:
%%time
# Train your estimator on S3 training data
estimator.fit({'train': train_location_orig_data})

2019-08-05 15:58:49 Starting - Starting the training job...
2019-08-05 15:58:51 Starting - Launching requested ML instances...
2019-08-05 15:59:21 Starting - Insufficient capacity error from EC2 while launching instances, retrying!......
2019-08-05 16:00:20 Starting - Preparing the instances for training......
2019-08-05 16:01:33 Downloading - Downloading input data.........
2019-08-05 16:03:04 Training - Downloading the training image..
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-08-05 16:03:23,390 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-08-05 16:03:23,414 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-08-05 16:03:29,640 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-08-05 16:03:29,864 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2019-0


2019-08-05 16:03:22 Training - Training image download completed. Training in progress.Using device cuda.
Get train data loader.
Epoch: 1 #011Training Loss: 3.712942 #011Validation Loss: 2.561787
Epoch: 2 #011Training Loss: 2.093467 #011Validation Loss: 1.663501
Epoch: 3 #011Training Loss: 1.476688 #011Validation Loss: 1.265527
Epoch: 4 #011Training Loss: 1.181037 #011Validation Loss: 1.050675
Epoch: 5 #011Training Loss: 1.011723 #011Validation Loss: 0.915356
Epoch: 6 #011Training Loss: 0.890234 #011Validation Loss: 0.823472
Epoch: 7 #011Training Loss: 0.804099 #011Validation Loss: 0.756152
Epoch: 8 #011Training Loss: 0.736964 #011Validation Loss: 0.703578
Epoch: 9 #011Training Loss: 0.688688 #011Validation Loss: 0.662970
Epoch: 10 #011Training Loss: 0.645880 #011Validation Loss: 0.631162
2019-08-05 16:18:31,413 sagemaker-containers INFO     Reporting training SUCCESS

2019-08-05 16:18:35 Uploading - Uploading generated training model
2019-08-05 16:19:51 Completed - Training job compl

In [11]:
%%time

# importing PyTorchModel
from sagemaker.pytorch import PyTorchModel

print(estimator.model_data)

# Create a model from the trained estimator data
# And point to the prediction script
model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='1.1.0',
                     entry_point='predict.py',
                     source_dir=source_dir)

s3://sagemaker-eu-central-1-505649883860/capstone-project-orig-data/sagemaker-pytorch-2019-08-05-15-58-48-712/output/model.tar.gz
CPU times: user 17.5 ms, sys: 46 µs, total: 17.5 ms
Wall time: 68.3 ms


In [12]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.large')

--------------------------------------------------------------------------------------------------------------!

In [17]:
testacc=test_acc(iter(dataloaders['test']), predictor)
print(testacc)

tensor([102,  12,   9, 105,  55])
preds is [102 111   9 105  60]
tensor([  4, 123, 106,   0, 132])
preds is [  4 123 106   0 126]
tensor([17,  0, 61, 48, 92])
preds is [17  0 61 48 92]
tensor([ 46, 123,  76,  22,  50])
preds is [ 46 123  76  22  50]
tensor([ 59,  50, 126,  30,   0])
preds is [ 59  50 126  30   0]
tensor([ 52,  98, 110,  44,  56])
preds is [ 75  98 110  44  56]
tensor([20, 85, 60, 21, 14])
preds is [20 85 76 21 14]
tensor([ 59,  10, 128, 108,   7])
preds is [ 59  10  50 108   7]
tensor([111,  22,  31, 129,  81])
preds is [111  22  31 129  81]
tensor([68, 30, 38, 45, 36])
preds is [68 30 38 76 36]
tensor([ 35, 109,  14,  81,  62])
preds is [ 35 117  14  81  67]
tensor([132,  10, 122,  28,   1])
preds is [132  10 122  28   1]
tensor([57, 99, 13, 81,  4])
preds is [57 99 13 81  4]
tensor([45, 46, 68, 55, 19])
preds is [45 46 68 55 19]
tensor([ 42,   3,   9, 103,  72])
preds is [ 42   3   9 103  36]
tensor([ 4, 93, 85, 45, 32])
preds is [ 4 93 85 45 73]
tensor([ 78,   4, 13

preds is [38 28 34 33 86]
tensor([ 23,   7,  26, 124, 115])
preds is [ 23   7  26 124 115]
tensor([ 76,  33, 118,  16, 110])
preds is [ 76  33 118  16 110]
tensor([110,  53,  25,  99,  74])
preds is [110  53  25  99 113]
tensor([100,  90,  15,  12, 105])
preds is [100  90  15  12 105]
tensor([ 82,  47,  70, 127,   0])
preds is [ 82 114  70 127   0]
tensor([77, 31, 74, 69,  1])
preds is [120  31  74 101  61]
tensor([ 58,  39,  36,  28, 119])
preds is [ 58  39  36  28 119]
tensor([ 91,  76,  95, 104,  43])
preds is [ 91  76  95 104  40]
tensor([ 0, 17, 86, 58, 43])
preds is [98 17 86 69  7]
tensor([13, 67, 97, 39, 79])
preds is [13 67 97 39 79]
tensor([  4,  19,  41, 109,  15])
preds is [  4  19  41 117  15]
tensor([ 70,  48,  37,  93, 111])
preds is [ 70  48  37  93 111]
tensor([ 48,  46,  39, 108, 117])
preds is [ 48  46  39 108 117]
tensor([117,  62, 116,  28, 103])
preds is [117  62 116  28 103]
tensor([56, 97, 74, 49,  5])
preds is [56 97 88 49  5]
tensor([119,   6, 111, 122,  15])
